# Trying Out Numba

In [1]:
import numba
from numba import jit
import dose_calculator_numba as dcn

In [15]:
%%time
n = 100000000

def add_a(n):
    a = 0
    b = 0
    for i in range(n):
        a += n
    for i in range(n):
        b += n
    return(a,b)


CPU times: user 17 µs, sys: 4 µs, total: 21 µs
Wall time: 36.7 µs


In [16]:
%%time
add_a(n)

CPU times: user 14.9 s, sys: 6.15 ms, total: 14.9 s
Wall time: 14.9 s


(10000000000000000, 10000000000000000)

In [17]:
%%time
@jit(nopython=True)
def add_a_fast(n):
    a = 0
    b = 0
    for i in range(n):
        a += n
    for i in range(n):
        b += n
    return(a,b)

CPU times: user 295 µs, sys: 36 µs, total: 331 µs
Wall time: 785 µs


In [25]:
%%time
add_a_fast(n)

CPU times: user 20 µs, sys: 2 µs, total: 22 µs
Wall time: 37.2 µs


(10000000000000000, 10000000000000000)

In [2]:
%%time 

# for a CT array of (Nx-1,Ny-1,Nz-1) voxels
Nx = 51
Ny = 51
Nz = 51

# distances between the x,y,z planes (also the lengths of the sides of the voxels) in cm
dx = 0.1
dy = 0.1
dz = 0.1

# initial and final coordinates of the beam
x1,x2 = (0,0)
y1,y2 = (0,0)
z1,z2 = (2.5,-2.5)

# adjustment from center 
adjust = 0.025

# initial plane coordinates
xplane1 = -2.5
yplane1 = -2.5
zplane1 = -2.5

# beam info and filename
beam_energy = 0.120 # in MeV
fluence_0 = 3.183098862 * 10**10 # photon/cm^2
# fluence_0 = 9.93 * 10**8 # photon/cm^2
filenames = ['energy_absorption_coeffs.txt','energy_absorption_coeffs_cort_bone.txt']

# kernel info
kernelname_water = '../Topas/Kernels/WaterKernel13.csv'
kernelname_bone = '../Topas/Kernels/BoneKernel3.csv'
kernelnames = [kernelname_water,kernelname_bone]
# kernelname = '../Topas/RealKernel1.csv'
kernel_size = (4,4,4) # cm 

# effective distance from center of kernel 
eff_dist = (2,2,2) # cm

# making materials array 
materials = []
for i in range(Nx-1):
    materials.append([])
    for j in range(Ny-1):
        materials[i].append([])
        for k in range(Nz-1):
            materials[i][j].append('w')

# number of cores to use
num_cores = 16

# Siddon((Nx,Ny,Nz),(dx,dy,dz),((x1,x2),(y1,y2),(z1,z2)),(xplane1,yplane1,zplane1),plot=True)
# terma = sd.TERMA((Nx,Ny,Nz),(dx,dy,dz),((x1,x2),(y1,y2),(z1,z2)),(xplane1,yplane1,zplane1),[beam_energy],[fluence_0],mu)

dose = dcn.Dose_Calculator((Nx,Ny,Nz),(dx,dy,dz),[((x1+adjust,x2+adjust),(y1+adjust,y2+adjust),(z1,z2)),((x1-adjust,x2-adjust),(y1+adjust,y2+adjust),(z1,z2)),((x1+adjust,x2+adjust),(y1-adjust,y2-adjust),(z1,z2)),((x1-adjust,x2-adjust),(y1-adjust,y2-adjust),(z1,z2))],(xplane1,yplane1,zplane1),[beam_energy],[fluence_0],filenames,kernelnames,kernel_size,eff_dist,materials,num_cores)


Calling Superposition


/home/user/NSERC-Summer-2022/Python/dose_calculator_numba.py:409: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "Superposition" failed type inference due to: Untyped global name 'sum': Cannot determine Numba type of <class 'builtin_function_or_method'>

File "dose_calculator_numba.py", line 573:
def Superposition(kernel_arrays,kernel_size,num_planes,voxel_lengths,voxel_info,beam_coor,eff_distance,mat_array,num_cores):
    <source elided>
        
        energy_deposit[ray] = np.array(sum(energy_deposit[ray]))
        ^

  @jit(parallel=True)
/home/user/.local/lib/python3.6/site-packages/numba/core/object_mode_passes.py:152: NumbaWarning: Function "Superposition" was compiled in object mode without forceobj=True.

File "dose_calculator_numba.py", line 410:
@jit(parallel=True)
def Superposition(kernel_arrays,kernel_size,num_planes,voxel_lengths,voxel_info,beam_coor,eff_distance,mat_array,num_cores):
^

  state.func_ir.loc))
/home/use

Kernel Interpolated


/home/user/NSERC-Summer-2022/Python/dose_calculator_numba.py:303: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "Superimpose" failed type inference due to: non-precise type pyobject
During: typing of argument at /home/user/NSERC-Summer-2022/Python/dose_calculator_numba.py (346)

File "dose_calculator_numba.py", line 346:
def Superimpose(voxel_info,voxel_array,kernel_func_water,kernel_func_bone,center_coor,kernel_coors_mat,kernel_value_water_total,kernel_value_bone_total,eff_voxels,mat_array):
    <source elided>
    
    energy_deposited = []
    ^

  @jit(parallel=True)


TypingError: Failed in object mode pipeline (step: object mode frontend)
Failed in object mode pipeline (step: convert make_function into JIT functions)
[1mCannot capture the non-constant value associated with variable 'kernel_func_bone' in a function that will escape.
[1m
File "dose_calculator_numba.py", line 350:[0m
[1m    
[1m    def kernel_func(ker_coors,CT_coors):
[0m    [1m^[0m[0m
[0m

This error may have been caused by the following argument(s):
- argument 0: [1mCannot determine Numba type of <class 'dict'>[0m
- argument 2: [1mCannot determine Numba type of <class 'scipy.interpolate.interpolate.RegularGridInterpolator'>[0m
- argument 3: [1mCannot determine Numba type of <class 'scipy.interpolate.interpolate.RegularGridInterpolator'>[0m
